In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import math
import os
import numpy as np
from collections import OrderedDict

In [2]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out		# 这里并没有使用到论文中的shared MLP, 而是直接相加了
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

In [3]:
class SEModule(nn.Module):
    expansion = 1

    def __init__(self, inplanes, stride=1, downsample=None):
        super(SEModule, self).__init__()
        self.conv1 = conv3x3(inplanes, inplanes, stride)
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(inplanes, inplanes)
        self.bn2 = nn.BatchNorm2d(inplanes)

        self.ca = ChannelAttention(inplanes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
       
#         out += residual
      
#         print(out.size())
        out = self.sa(out) * out
#         print(out.size())
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
#         print(out.size())
        out = self.relu(out)

        return out

In [4]:
def conv31(ip,op,stride=1):
    return nn.Conv2d(ip,op,kernel_size=3,stride=stride,padding=1,bias=False)
def conv32(ip,op,stride=2):
    return nn.Conv2d(ip,op,kernel_size=3,stride=stride,padding=1,bias=False)
class Upsample(nn.Module):
    def __init__(self,ip,op):
        super(Upsample,self).__init__()
        self.upsample=nn.Sequential(
            conv31(ip,op,1),
            nn.Upsample(scale_factor=2,mode='nearest')
        )
        
    def forward(self,x):
        x=self.upsample(x)
        return x

In [5]:
def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        h_swish()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        h_swish()
    )
def conv2d(filter_in, filter_out, kernel_size, groups=1, stride=1):
    pad = (kernel_size - 1) // 2 if kernel_size else 0
    return nn.Sequential(OrderedDict([
        ("conv", nn.Conv2d(filter_in, filter_out, kernel_size=kernel_size, stride=stride, padding=pad, groups=groups, bias=False)),
        ("bn", nn.BatchNorm2d(filter_out)),
        ("relu", nn.ReLU6(inplace=True)),
    ]))

def conv_dw(filter_in, filter_out, stride = 1):
    return nn.Sequential(
        nn.Conv2d(filter_in, filter_in, 3, stride, 1, groups=filter_in, bias=False),
        nn.BatchNorm2d(filter_in),
        nn.ReLU6(inplace=True),

        nn.Conv2d(filter_in, filter_out, 1, 1, 0, bias=False),
        nn.BatchNorm2d(filter_out),
        nn.ReLU6(inplace=True),
    )
#---------------------------------------------------#
#   Èý´Î¾í»ý¿é
#---------------------------------------------------#
def make_three_conv(filters_list, in_filters):
    m = nn.Sequential(
        conv2d(in_filters, filters_list[0], 1),
        conv_dw(filters_list[0], filters_list[1]),
        conv2d(filters_list[1], filters_list[0], 1),
    )
    return m

#---------------------------------------------------#
#   Îå´Î¾í»ý¿é
#---------------------------------------------------#
def make_five_conv(filters_list, in_filters):
    m = nn.Sequential(
        conv2d(in_filters, filters_list[0], 1),
        conv_dw(filters_list[0], filters_list[1]),
        conv2d(filters_list[1], filters_list[0], 1),
        conv_dw(filters_list[0], filters_list[1]),
        conv2d(filters_list[1], filters_list[0], 1),
    )
    return m

In [6]:
def Hswish(x,inplace=True):
    return x * F.relu6(x + 3., inplace=inplace) / 6.

def Hsigmoid(x,inplace=True):
    return F.relu6(x + 3., inplace=inplace) / 6.


class Bottleneck(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,exp_channels,stride,se='True',nl='HS'):
        super(Bottleneck, self).__init__()
        padding = (kernel_size - 1) // 2
        if nl == 'RE':
            self.nlin_layer = F.relu6
        elif nl == 'HS':
            self.nlin_layer = Hswish
        self.stride=stride
        if se:
            self.se=SEModule(exp_channels)
        else:
            self.se=None
        self.conv1=nn.Conv2d(in_channels,exp_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn1 = nn.BatchNorm2d(exp_channels)
        self.conv2=nn.Conv2d(exp_channels,exp_channels,kernel_size=kernel_size,stride=stride,
                             padding=padding,groups=exp_channels,bias=False)
        self.bn2=nn.BatchNorm2d(exp_channels)
        self.conv3=nn.Conv2d(exp_channels,out_channels,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn3=nn.BatchNorm2d(out_channels)
        # ÏÈ³õÊ¼»¯Ò»¸ö¿ÕÐòÁÐ£¬Ö®ºó¸ÄÔìÆä³ÉÎª²Ð²îÁ´½Ó
        self.shortcut = nn.Sequential()
        # Ö»ÓÐ²½³¤Îª1ÇÒÊäÈëÊä³öÍ¨µÀ²»ÏàÍ¬Ê±²Å²ÉÓÃÌøÔ¾Á¬½Ó(ÏëÒ»ÏÂÌøÔ¾Á´½ÓµÄ¹ý³Ì£¬ÊäÈëÊä³öÍ¨µÀÏàÍ¬Õâ¸öÌøÔ¾Á¬½Ó¾ÍÃ»ÒâÒåÁË)
        if stride == 1 and in_channels != out_channels:
            self.shortcut = nn.Sequential(
                # ÏÂÃæµÄ²Ù×÷¾í»ý²»¸Ä±ä³ß´çºÍÍ¨µÀÊý
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self,x):
        out=self.nlin_layer(self.bn1(self.conv1(x)))
        if self.se is not None:
            out=self.bn2(self.conv2(out))
            out=self.nlin_layer(self.se(out))
        else:
            out = self.nlin_layer(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride == 1 else out
        return out


class MobileNetV3_large(nn.Module):
    # (out_channels,kernel_size,exp_channels,stride,se,nl)
    cfg=[
        (16,3,16,1,False,'RE'),
        (24,3,64,2,False,'RE'),
        (24,3,72,1,False,'RE'),
        (40,5,72,2,True,'RE'),
        (40,5,120,1,True,'RE'),
        (40,5,120,1,True,'RE'),
        (80,3,240,2,False,'HS'),
        (80,3,200,1,False,'HS'),
        (80,3,184,1,False,'HS'),
        (80,3,184,1,False,'HS'),
        (112,3,480,1,True,'HS'),
        (112,3,672,1,True,'HS'),
        (160,5,672,2,True,'HS'),
        (160,5,960,1,True,'HS'),
        (160,5,960,1,True,'HS')
    ]
    def __init__(self,num_classes=5):
        super(MobileNetV3_large,self).__init__()
        self.conv1=nn.Conv2d(3,16,3,2,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(16)
        # ¸ù¾ÝcfgÊý×é×Ô¶¯Éú³ÉËùÓÐµÄBottleneck²ã
        self.layers = self._make_layers(in_channels=16)
        self.conv2=nn.Conv2d(160,960,1,stride=1,bias=False)
        self.bn2=nn.BatchNorm2d(960)
        # ¾í»ýºó²»¸úBN£¬¾ÍÓ¦¸Ã°ÑbiasÉèÖÃÎªTrue
        self.conv3=nn.Conv2d(960,1280,1,1,padding=0,bias=True)
        self.conv4=nn.Conv2d(1280,num_classes,1,stride=1,padding=0,bias=True)
        
        self.down_sample1 = conv_dw(40,112,stride=2)
        self.mk1=make_five_conv([112,224],224)
        
        self.down_sample2 = conv_dw(112,160,stride=2)
        self.mk2=make_five_conv([160,320],320)
        
#         self.w=nn.Parameter(torch.FloatTensor(1),requires_grad=True)
#         self.w=torch.FloatTensor(1)
        self.weights_try=1
        self.weights2_try=1
        self.weights3_try=1
        
#         self.w.data.fill_(1)
#         self.weights2_try.data.fill_(1)
#         self.weights3_try.data.fill_(1)
        
    def _make_layers(self,in_channels):
        layers=[]
        for out_channels,kernel_size,exp_channels,stride,se,nl in self.cfg:
            layers.append(
                Bottleneck(in_channels,out_channels,kernel_size,exp_channels,stride,se,nl)
            )
            in_channels=out_channels
        return nn.Sequential(*layers)
                

    def forward(self,x):
        out=Hswish(self.bn1(self.conv1(x)))
        ##############################################
        out1=self.layers[:6](out) # 28*28*40
        out2=self.layers[6:12](out1) # 14*14*112
        out3=self.layers[12:16](out2) # 7*7*160
        
        out1_dw=self.down_sample1(out1)
        out2=torch.cat([out1_dw,out2],1)
        out2=self.mk1(out2)
        out2_dw=self.down_sample2(out2)
        out3=torch.cat([out2_dw,out3],1)
        out3=self.mk2(out3)
        
        
        
        
        out=self.layers(out)
        out=out+out3
        out=Hswish(self.bn2(self.conv2(out)))
        out=F.avg_pool2d(out,7)
        out=Hswish(self.conv3(out))
        out=self.conv4(out)
        
        a,b=out.size(0),out.size(1)
        out=out.view(a,b)
#         print(self.weights_try)
#         return out,out1,out2,out3
        return out

In [7]:
model=MobileNetV3_large()

In [8]:
# x=torch.randn(2,3,224,224)
# y=model(x)
# print(y.size())
# print(y)

In [9]:
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader

In [10]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset

random.seed(1)

class flowerDataset(Dataset):
    # 自定义Dataset类，必须继承Dataset并重写__init__和__getitem__函数
    def __init__(self, data_dir, transform=None):
        """
            花朵分类任务的Dataset
            :param data_dir: str, 数据集所在路径
            :param transform: torch.transform，数据预处理，默认不进行预处理
        """
        # data_info存储所有图片路径和标签（元组的列表），在DataLoader中通过index读取样本
        self.data_info = self.get_img_info(data_dir)
        self.transform = transform

    def __getitem__(self, index):
        path_img, label = self.data_info[index]
        # 打开图片，默认为PIL，需要转成RGB
        img = Image.open(path_img).convert('RGB')
        # 如果预处理的条件不为空，应该进行预处理操作
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data_info)

    # 自定义方法，用于返回所有图片的路径以及标签
    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            # 遍历类别
            for sub_dir in dirs:
                # listdir为列出文件夹下所有文件和文件夹名
                img_names = os.listdir(os.path.join(root, sub_dir))
                # 过滤出所有后缀名为jpg的文件名（那当然也就把文件夹过滤掉了）
#                 img_names = list(filter(lambda x: x.endswith('.png'), img_names))

                # 遍历图片
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    # 在该任务中，文件夹名等于标签名
                    label = sub_dir
                    data_info.append((path_img, int(label)))
        return data_info

In [11]:
#宏定义一些数据，如epoch数，batchsize等
MAX_EPOCH=45
BATCH_SIZE=64
# LR=0.0001
LR=0.001
log_interval=30
val_interval=1

In [12]:
# ============================ step 1/5 数据 ============================
# split_dir=os.path.join("MobileNetV3-master",".","data","splitData")
split_dir=r'/home/fanrz/Desktop/torchmob/'
train_dir=r'/home/fanrz/Desktop/testlarge/aptos/tra2/'
valid_dir=os.path.join(split_dir,"val")

In [13]:
#对训练集所需要做的预处理
train_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [14]:
#对验证集所需要做的预处理
valid_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [15]:
# 构建MyDataset实例
train_data=flowerDataset(data_dir=train_dir,transform=train_transform)
valid_data=flowerDataset(data_dir=valid_dir,transform=valid_transform)

In [16]:
print(len(train_data.data_info))

5035


In [17]:
# 构建DataLoader
# 训练集数据最好打乱
# DataLoader的实质就是把数据集加上一个索引号，再返回
train_loader=DataLoader(dataset=train_data,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True
                       )
valid_loader=DataLoader(dataset=valid_data,batch_size=BATCH_SIZE,drop_last=True)

In [18]:
# ============================ step 2/5 模型 ============================
if torch.cuda.is_available():
    model=nn.DataParallel(model)
    model.cuda()

In [19]:
# ============================ step 3/5 损失函数 ============================
criterion=nn.CrossEntropyLoss()
# ============================ step 4/5 优化器 ============================
optimizer=optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.99))# 选择优化器

# optimizer2=optim.Adam(model.parameters(),lr=LR, betas=(0.9, 0.99))# 选择优化器

In [20]:
def gs(output,label):
    s=0.0
    for i,n in enumerate(label):
        s+=output[i][n]
    return s/(i+1)
def gw(weights):
    s=0
    for i in weights:
        if(i>0):
            s+=i
    return s
def cw(grad_block,batch):
    count_weight_list=[]
    i=0
    while(i<batch):
        grads_val=grad_block[0][i].cpu().data.numpy().squeeze()
        grads=grads_val.reshape(grads_val.shape[0],-1)
        weights=np.mean(grads,axis=1)
        w=gw(weights)
        count_weight_list.append(w)
        grads=0
        grads_val=0
        i+=1
    return np.mean(count_weight_list)
def count_final(grad_block,grad_block2,grad_block3,batch_size):
    score_last=cw(grad_block,batch_size)
    score_mid=cw(grad_block2,batch_size)
    score_first=cw(grad_block3,batch_size)
    w_last=score_last/(score_last+score_first+score_mid)
    w_mid=score_mid/(score_last+score_first+score_mid)
    w_first=score_first/(score_last+score_first+score_mid)
    return w_first,w_mid,w_last

In [21]:
# ============================ step 5/5 训练 ============================
# 记录每一次的数据，方便绘图
train_curve=list()
valid_curve=list()
model.train()
accurancy_global=0.0
# model.layers[-1].register_forward_hook(farward_hook)
# model.layers[-1].register_backward_hook(backward_hook)
# model.layers[-5].register_backward_hook(backward2_hook)
# model.layers[-11].register_backward_hook(backward3_hook)
for epoch in range(MAX_EPOCH):
    loss_mean=0.
    correct=0.
    total=0.
    running_loss = 0.0

    for i,data in enumerate(train_loader):
        
        img,label=data
        img = Variable(img)
        label = Variable(label)
        if torch.cuda.is_available():
            img=img.cuda()
            label=label.cuda()
        
        out=model(img)
        
#         optimizer2.zero_grad()
#         class_loss=gs(out,label)
#         class_loss.backward(retain_graph=True)
        
#         w_f,w_m,w_l=count_final(grad_block,grad_block2,grad_block3,BATCH_SIZE)
        
            
#         grad_block=[]
#         grad_block2=[]
#         grad_block3=[]
        
        optimizer.zero_grad()  # 归0梯度
        ###########################################
#         print_loss=loss.data.item()
        ###########################################
        
        loss=criterion(out,label)#得到损失函数
        
#         class_loss.backward(retain_graph=True)
        ###########################################
        loss.backward()#反向传播
        ###########################################
        optimizer.step()#优化
        
        
        if (i+1)%log_interval==0:
            print('epoch:{},loss:{:.4f}'.format(epoch+1,loss.data.item()))
        _, predicted = torch.max(out.data, 1)
        total += label.size(0)
#         if (i+1)%log_interval==0:
#             print("============================================")
#             print("源数据标签：",label)
#             print("============================================")
#             print("预测结果：",predicted)
#             print("相等的结果为：",predicted == label)
        correct += (predicted == label).sum()
        if (i+1)%log_interval==0:
            print(correct.item() / total)
#         print(correct.item())

#     fi=epoch
#     if (w_l>0.7)and(fi==0):
#         fi+=1
#         model.weights_try=w_f
#         model.weights2_try=w_m
#         model.weights3_try=w_l
#         print(w_f,w_m,w_l)
#         print('固定参数')
#         print(fi)
#     model.weights_try=w_f
#     model.weights2_try=w_m
#     model.weights3_try=w_l
    print("============================================")
#     print(w_f,w_m,w_l)
    accurancy=correct.item() / total
    if accurancy>accurancy_global:
        torch.save(model.state_dict(), './weights/best330.pkl')
        print("准确率由：", accurancy_global, "上升至：", accurancy, "已更新并保存权值为weights/best.pkl")
        accurancy_global=accurancy
    print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, 100*accurancy))
# torch.save(model.state_dict(), './weights/last322.pkl')
print("训练完毕，权重已保存为：weights/last.pkl")


epoch:1,loss:1.2188
0.4005208333333333
epoch:1,loss:1.0881
0.44765625
准确率由： 0.0 上升至： 0.453125 已更新并保存权值为weights/best.pkl
第1个epoch的识别准确率为：45%
epoch:2,loss:1.1982
0.5223958333333333
epoch:2,loss:1.1389
0.51328125
准确率由： 0.453125 上升至： 0.5198317307692307 已更新并保存权值为weights/best.pkl
第2个epoch的识别准确率为：51%
epoch:3,loss:1.1810
0.553125
epoch:3,loss:1.0320
0.5463541666666667
准确率由： 0.5198317307692307 上升至： 0.5406650641025641 已更新并保存权值为weights/best.pkl
第3个epoch的识别准确率为：54%
epoch:4,loss:1.0129
0.5578125
epoch:4,loss:0.9625
0.5638020833333334
准确率由： 0.5406650641025641 上升至： 0.5592948717948718 已更新并保存权值为weights/best.pkl
第4个epoch的识别准确率为：55%
epoch:5,loss:1.2423
0.5708333333333333
epoch:5,loss:1.0992
0.57578125
准确率由： 0.5592948717948718 上升至： 0.5725160256410257 已更新并保存权值为weights/best.pkl
第5个epoch的识别准确率为：57%
epoch:6,loss:1.0010
0.6067708333333334
epoch:6,loss:0.9287
0.6057291666666667
准确率由： 0.5725160256410257 上升至： 0.5971554487179487 已更新并保存权值为weights/best.pkl
第6个epoch的识别准确率为：59%
epoch:7,loss:0.9492
0.6072916666666667
e

第43个epoch的识别准确率为：96%
epoch:44,loss:0.1102
0.9666666666666667
epoch:44,loss:0.1641
0.9604166666666667
第44个epoch的识别准确率为：95%
epoch:45,loss:0.0363
0.9760416666666667
epoch:45,loss:0.2120
0.9682291666666667
准确率由： 0.9659455128205128 上升至： 0.9689503205128205 已更新并保存权值为weights/best.pkl
第45个epoch的识别准确率为：96%
训练完毕，权重已保存为：weights/last.pkl


In [22]:
torch.__version__

'1.0.0'